# Дообучение моделей с Hugging Face

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

## Работа с датасетами



Здесь мы загружаем исходные данные, как-либо преобразуем их к удобоваримому виду ['text', 'labels'] и записываем датафрейс в переменную train_dataset. Её, по желанию, можно сохранить в папку data/forfinetuning.

В датафрейме labels был NEUTRAL / POSITIVE / NEGATIVE

### Работа с начальными данными

#### Сравнятор

['netologiya',
 'geekbrains',
 'skillfactory',
 'hexlet',
 'stepik',
 'brunoyam',
 'otus',
 'elbrus-coding-bootcamp',
 'skypro',
 'yandeks-praktikum',
 'html-academy',
 'qa-studio',
 'software-testing-ru',
 'slerm',
 'karpov-courses',
 'xyz-school',
 'contented',
 'productstar',
 'uchebnik',
 'gopractice',
 'umskul',
 'skillbox']

 razrabotka
 dizayn
 analitika
 management
 english

In [ ]:
sravnyator = pd.read_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/feedbacks_sravnyator_new.csv").drop("idx", axis=1)
sravnyator

,rating,short_name,dignities,disadvantages,impressions
0,5,Понятное изложение информации и большое количе...,Удобство занятий онлайн в любое свободное врем...,Платформа работала стабильно. Нареканий к курс...,"Курс посоветовал бы тем, кто хочет найти работ..."
1,5,Организаторы всегда идут навстречу обучающимся,"Хороший курс, если хочется узнать больше о тес...",Отмечу небольшой оргмомент: в блоке автотестир...,Хочу сказать огромное спасибо очень коммуникаб...
2,5,"Стоит своих денег, но придется попотеть","Лекции проводятся в онлайн-формате, есть чатик...",Все отлично.,"Курс однозначно стоит своих денег, но не стоит..."
3,5,C такими крутыми преподавателями хочется учить...,"Мне очень понравилась атмосфера, с которой был...",Быстро закончился курс. Хочется еще учиться с ...,"Ожидания оправдались, на середине курса я труд..."
4,5,Ни разу не пожалел о своем выборе,— Живые лекции;— опытные лекторы;— интересные ...,Не хранится история общего чата.,"Первое, что хотелось бы отметить, это преподав..."
...,...,...,...,...,...
490,2,"Бессмысленный курс, учат не тому и не так",Демоуроки и несколько первых лекций выглядят н...,"Структура курса не продумана, не дает полной к...","Проходила курс моя девушка, я наблюдал за этим..."
491,1,Не увидел разницы между уроками на курсе и уро...,Оценил скорость обратной связи специалистов. П...,"Говоря простыми словами, не увидел для себя ос...","Я думаю, что, во-первых, прежде чем продавать ..."
492,5,"Все нравится, все устраивает","Отличный курс, позволяет влиться в тему 1С даж...",Первые несколько уроков читает другой преподав...,"В данный момент прохожу курс, на 15 модуле из ..."
493,1,"Образовательного материала не хватает, а разор...",Их нет.,Образовательной информации хватать не будет. Э...,"Проходить мимо, почитав отзывы от таких наивны..."


In [ ]:
sravnyator_sentiments = pd.concat([
    pd.DataFrame({
        "text": sravnyator[sravnyator.dignities.apply(lambda x: len(x) > 60) & sravnyator.rating.apply(lambda x: x in [5, 4])].dignities,
        "labels": 1
    }),
    pd.DataFrame({
        "text": sravnyator[sravnyator.disadvantages.apply(lambda x: len(x) > 60) & sravnyator.rating.apply(lambda x: x in [1, 2])].disadvantages,
        "labels": 2
    }),
    # new
    pd.DataFrame({
        "text": sravnyator[sravnyator.disadvantages.apply(lambda x: len(x) > 60) & sravnyator.rating.apply(lambda x: x == 3)].disadvantages,
        "labels": 0
    })
])

print(sravnyator_sentiments.labels.value_counts())
sravnyator_sentiments

1    406
2     47
0     26
Name: labels, dtype: int64


,text,labels
0,Удобство занятий онлайн в любое свободное врем...,1
1,"Хороший курс, если хочется узнать больше о тес...",1
2,"Лекции проводятся в онлайн-формате, есть чатик...",1
3,"Мне очень понравилась атмосфера, с которой был...",1
5,Мне понравилась манера подачи информации. Мате...,1
...,...,...
476,"1. Периодически зависают видео с обучением, но...",0
479,Очень поверхностная и сильно сжатая информация...,0
487,Видеоуроки зачастую не содержат никаких конспе...,0
489,Я выбрала курс «Frontend-разработчик с нуля до...,0


In [ ]:
sravnyator["short_plus_impressions"] = sravnyator["short_name"] + ". " + sravnyator["impressions"]
sravnyator

,rating,short_name,dignities,disadvantages,impressions,short_plus_impressions
0,5,Понятное изложение информации и большое количе...,Удобство занятий онлайн в любое свободное врем...,Платформа работала стабильно. Нареканий к курс...,"Курс посоветовал бы тем, кто хочет найти работ...",Понятное изложение информации и большое количе...
1,5,Организаторы всегда идут навстречу обучающимся,"Хороший курс, если хочется узнать больше о тес...",Отмечу небольшой оргмомент: в блоке автотестир...,Хочу сказать огромное спасибо очень коммуникаб...,Организаторы всегда идут навстречу обучающимся...
2,5,"Стоит своих денег, но придется попотеть","Лекции проводятся в онлайн-формате, есть чатик...",Все отлично.,"Курс однозначно стоит своих денег, но не стоит...","Стоит своих денег, но придется попотеть. Курс ..."
3,5,C такими крутыми преподавателями хочется учить...,"Мне очень понравилась атмосфера, с которой был...",Быстро закончился курс. Хочется еще учиться с ...,"Ожидания оправдались, на середине курса я труд...",C такими крутыми преподавателями хочется учить...
4,5,Ни разу не пожалел о своем выборе,— Живые лекции;— опытные лекторы;— интересные ...,Не хранится история общего чата.,"Первое, что хотелось бы отметить, это преподав...","Ни разу не пожалел о своем выборе. Первое, что..."
...,...,...,...,...,...,...
490,2,"Бессмысленный курс, учат не тому и не так",Демоуроки и несколько первых лекций выглядят н...,"Структура курса не продумана, не дает полной к...","Проходила курс моя девушка, я наблюдал за этим...","Бессмысленный курс, учат не тому и не так. Про..."
491,1,Не увидел разницы между уроками на курсе и уро...,Оценил скорость обратной связи специалистов. П...,"Говоря простыми словами, не увидел для себя ос...","Я думаю, что, во-первых, прежде чем продавать ...",Не увидел разницы между уроками на курсе и уро...
492,5,"Все нравится, все устраивает","Отличный курс, позволяет влиться в тему 1С даж...",Первые несколько уроков читает другой преподав...,"В данный момент прохожу курс, на 15 модуле из ...","Все нравится, все устраивает. В данный момент ..."
493,1,"Образовательного материала не хватает, а разор...",Их нет.,Образовательной информации хватать не будет. Э...,"Проходить мимо, почитав отзывы от таких наивны...","Образовательного материала не хватает, а разор..."


In [ ]:
sravnyator_stars = pd.concat([
    pd.DataFrame({
        "text": sravnyator[sravnyator.rating.apply(lambda x: x == 5)].short_plus_impressions.reset_index(drop=True),
        "labels": 1
    }),
    pd.DataFrame({
        "text": sravnyator[sravnyator.rating.apply(lambda x: x in [1, 2])].short_plus_impressions.reset_index(drop=True),
        "labels": 2
    }),
    # new
    pd.DataFrame({
        "text": sravnyator[sravnyator.rating.apply(lambda x: x == 3)].short_plus_impressions.reset_index(drop=True),
        "labels": 0
    })
])

print(sravnyator_stars.labels.value_counts())
sravnyator_stars

1    345
2     52
0     27
Name: labels, dtype: int64


,text,labels
0,Понятное изложение информации и большое количе...,1
1,Организаторы всегда идут навстречу обучающимся...,1
2,"Стоит своих денег, но придется попотеть. Курс ...",1
3,C такими крутыми преподавателями хочется учить...,1
4,"Ни разу не пожалел о своем выборе. Первое, что...",1
...,...,...
22,"Можно попробовать, если вы хоть что-то знаете ...",0
23,Курс до сих пор не записан до конца. Ожиданий ...,0
24,Дарит иллюзорную надежду на смену профессии. Д...,0
25,В проданном курсе не оказалось нужного мне мод...,0


#### Курсатор

In [ ]:
labels = { -1 : 2, 0 : 0, 1 : 1 }

feedback_coursator = pd.read_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/feedback_coursator.csv")[['text', 'sentiment']].dropna()
feedback_coursator.columns = ["text", "labels"]
feedback_coursator = feedback_coursator.replace({'labels' : labels})

print(feedback_coursator.labels.value_counts())
feedback_coursator

1    1535
2     280
0      53
Name: labels, dtype: int64


,text,labels
0,У сына начались проблемы с математикой. Оплати...,1
1,Я сейчас в декрете. Решила получить новую проф...,1
2,Дочь 8 лет занимается китайским языком. Раньше...,1
3,Хотела заняться парковым дизайном. Компания не...,2
4,Можно сказать заново изучала обществознание. П...,1
...,...,...
1863,На курс этот вышла с видео на ютубчике. Гаврил...,2
1864,Долго выбирала интернет-курсы для прокачки моз...,1
1865,"Я очень долго думала, чем могу заниматься. Род...",1
1866,"У меня жизнь была яркой, много работы, впечатл...",1


In [ ]:
feedback_coursator = feedback_coursator.sample(frac=1).groupby("labels").head(360)
feedback_coursator.labels.value_counts()

1    360
2    280
0     53
Name: labels, dtype: int64

#### МУПЫ

In [ ]:
elective_mups = pd.read_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/another/_electives_descriptions/electives.csv").dropna().description
elective_mups = pd.DataFrame({"text": elective_mups, "labels": 0})
elective_mups

,text,labels
0,Основная проблематика курса включает:\n– Разв...,0
1,Курс «История книги и чтения» знакомит студент...,0
2,Изучение истории немецкой литературы будет про...,0
3,"Данный курс является частью minor \""Франция: и...",0
4,Курс «История французской литературы» построен...,0
...,...,...
437,Основные разделы курса будут посвящены составл...,0
438,Цель курса - изучение и освоение разных подход...,0
439,В рамках курса исследуются основные характерис...,0
440,В процессе обучения будут изучены следующие ра...,0


In [ ]:
# another_mups = pd.read_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/moops_modeus.csv.gz", compression="gzip").dropna().description.reset_index(drop=True)
# another_mups = pd.DataFrame({"text": another_mups, "labels": 0})
# another_mups = another_mups.drop_duplicates().reset_index(drop=True)
# another_mups = another_mups[~another_mups.apply(tuple, 1).isin(elective_mups.apply(tuple, 1))]
# another_mups

,text,labels
0,Цель курса – дать общее понимание репрезентати...,0
1,"Дисциплина «Асимптотические методы», предназна...",0
2,"Дисциплина ""Асимптотические методы"" изучается ...",0
3,Целью курса «Астрономия» является изложение ос...,0
4,Целью курса «Астрофизика» являются изложение о...,0
...,...,...
11533,\tЦель курса — повышение профессионального уро...,0
11534,Цель курса — формирование знаний и представлен...,0
11535,"\tЦель дисциплины: изучение истории развития, ...",0
11536,"Цель дисциплины: изучение истории развития, со...",0


In [ ]:
# all_mups = pd.concat([mups, another_mups.sample(600)])
# all_mups

,text,labels
0,Основная проблематика курса включает:\n– Разв...,0
1,Курс «История книги и чтения» знакомит студент...,0
2,Изучение истории немецкой литературы будет про...,0
3,"Данный курс является частью minor \""Франция: и...",0
4,Курс «История французской литературы» построен...,0
...,...,...
4611,Благодаря социальной практике осуществляется п...,0
6099,Учебная дисциплина Проблемы судебного производ...,0
9168,Цель ­– получение знаний о математических ме...,0
11271,Вид практики –преддипломная.Способ – стационар...,0


#### Типичный абитуриент

In [ ]:
tabiturient = pd.read_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/tabiturient.csv").dropna().drop("Unnamed: 0", axis=1)

print(tabiturient.labels.value_counts())
tabiturient

2    134
1     97
0     44
Name: labels, dtype: int64


,text,labels
0,"Приветсвую вас, уважаемые читатели! Этим л...",1
1,Дорогие и многоуважаемые абитуриенты! Эти р...,2
2,МЭИ. Институт информационных и вычислительн...,2
3,"Надеюсь, что мой отзыв увидит как можно бол...",2
4,"РАНХиГС, ИОН (Институт общественных наук), ...",1
...,...,...
270,Я студент МЖ-МИЭП 4 курса. МГИМО - это сове...,2
271,2018-2019 Это худший на данное время медици...,2
272,Являясь выпускником этого вуза и пройдя чер...,2
273,"Окончила филфак пару лет назад, сейчас пони...",2


In [ ]:
tabiturient = tabiturient.sample(frac=1).groupby("labels").head(100)
tabiturient.labels.value_counts()

2    100
1     97
0     44
Name: labels, dtype: int64

### Собираем ВСЁЁЁ в один датафрейм

Разбиваем тексты на мини-тексты по 8 предложений и, если надо, сохраняем/выгружаем

In [ ]:
solyanka = pd.concat([elective_mups,
                      sravnyator_sentiments,
                      sravnyator_stars,
                      feedback_coursator,
                      tabiturient,
                      tabiturient[tabiturient.labels.apply(lambda x: x == 0)]]).dropna()

print(solyanka.labels.value_counts())
solyanka

1    1208
0     593
2     479
Name: labels, dtype: int64


,text,labels
0,Основная проблематика курса включает:\n– Разв...,0
1,Курс «История книги и чтения» знакомит студент...,0
2,Изучение истории немецкой литературы будет про...,0
3,"Данный курс является частью minor \""Франция: и...",0
4,Курс «История французской литературы» построен...,0
...,...,...
134,Поступал сюда в прошлом году. Прошел как го...,0
105,"Я существую — я студентка, которая к своему...",0
210,"Вот и подошёл к концу 3 семестр, а значит, ...",0
152,"Доброго времени суток, друзья. Я учусь на п...",0


In [ ]:
!pip install razdel
from razdel import sentenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def getSentences(x, n_sentences=8):
  sentences = [i.text for i in sentenize(x.replace("\n", " ").replace("\t", " ").replace("\r", ""))]
  parts = [""] * (len(sentences) // n_sentences + 1)
  part = ""

  for i in range(len(sentences)):
    parts[i // n_sentences] += " ".join(sentences[i].split()[:75]) + " "

  return [i for i in parts if len(i) > 2]

In [ ]:
getSentences("Стоят пассажиры в аэропорту, посадочный досмотр проходят. Дошла очередь до мужика с чемоданом. — Пожалуйста, приготовьте чемодан к осмотру. — Не могу. — Почему? — А у меня там бипки! — А что это? — Ну, отсосёте — покажу! Служба безопасности шутку не поняла, вызвала наряд полиции: — Гражданин, открывайте чемодан. — Но я не могу! — Почему не можете? — Потому что у меня там бипки! — «Бипки»? Что это? — Так отсосите мне, и я покажу! Отобрали чемодан, а открыть никто не может. Повезли мужика в СИЗО. В камере сидельцы расспрашивают: — Братуха, за что тебя? — Да чемодан отказался открывать. — А что там? — Да бипки там. — Какие нахуй «бипки»? — Ну отсосите мне, тогда и расскажу. И вот угодил мужик в лазарет, с побоями да синяками по всему телу, весь перебинтован, еле дышит. Следователи вызвали группу специалистов для вскрытия чемодана. Час, два, три пыхтели. Кое-как разворотили, смотрят — а там бипки.")

['Стоят пассажиры в аэропорту, посадочный досмотр проходят. Дошла очередь до мужика с чемоданом. — Пожалуйста, приготовьте чемодан к осмотру. — Не могу. — Почему? — А у меня там бипки! — А что это? — Ну, отсосёте — покажу! ',
 'Служба безопасности шутку не поняла, вызвала наряд полиции: — Гражданин, открывайте чемодан. — Но я не могу! — Почему не можете? — Потому что у меня там бипки! — «Бипки»? Что это? — Так отсосите мне, и я покажу! Отобрали чемодан, а открыть никто не может. ',
 'Повезли мужика в СИЗО. В камере сидельцы расспрашивают: — Братуха, за что тебя? — Да чемодан отказался открывать. — А что там? — Да бипки там. — Какие нахуй «бипки»? — Ну отсосите мне, тогда и расскажу. И вот угодил мужик в лазарет, с побоями да синяками по всему телу, весь перебинтован, еле дышит. ',
 'Следователи вызвали группу специалистов для вскрытия чемодана. Час, два, три пыхтели. Кое-как разворотили, смотрят — а там бипки. ']

In [ ]:
solyanka_razdeled = {"text": [], "labels": []}
for i in solyanka.to_numpy():
  sents = getSentences(i[0])
  for j in sents:
    solyanka_razdeled["text"].append(j)
    solyanka_razdeled["labels"].append(i[1])
solyanka_razdeled = pd.DataFrame(solyanka_razdeled)

print(solyanka_razdeled.labels.value_counts())
solyanka_razdeled

1    2059
0    1489
2    1255
Name: labels, dtype: int64


,text,labels
0,Основная проблематика курса включает:\n– Разви...,0
1,Курс «История книги и чтения» знакомит студент...,0
2,Изучение истории немецкой литературы будет про...,0
3,"Данный курс является частью minor \""Франция: и...",0
4,Курс «История французской литературы» построен...,0
...,...,...
4798,А теперь по общим темам: Средний балл: в МИФИ ...,0
4799,"Запомнили. Те, кто не пройдет, извлекайте плюс...",0
4800,"И последний – КФС: купоны наше все, но только ...",0
4801,"Паблики в ВК – разношерстная компания, поэтому...",0


In [ ]:
train_df = pd.concat([solyanka_razdeled,
                               solyanka_razdeled[solyanka_razdeled.labels.apply(lambda x: x == 0)].sample(500),
                               solyanka_razdeled[solyanka_razdeled.labels.apply(lambda x: x == 2)].sample(700)
                               ]).reset_index(drop=True)

print(train_df.labels.value_counts())
train_df

1    2059
0    1989
2    1955
Name: labels, dtype: int64


,text,labels
0,Основная проблематика курса включает:\n– Разви...,0
1,Курс «История книги и чтения» знакомит студент...,0
2,Изучение истории немецкой литературы будет про...,0
3,"Данный курс является частью minor \""Франция: и...",0
4,Курс «История французской литературы» построен...,0
...,...,...
5998,"Может быть, какому-нибудь студенту не так жаль...",2
5999,"Общага коридорного типа, так что о своих кухне...",2
6000,Начитавшись отзывов о факультетах МО в разных ...,2
6001,"На лекциях творится Адъ и Израиль, лектора ник...",2


In [ ]:
train_df.to_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/forfinetuning/solyanka_parted.csv", index=False)

## Дообучение

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset, DatasetDict, Features, Value, ClassLabel, Dataset

### Сбор train+valid датасета

In [ ]:
# train_df = pd.read_csv("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/forfinetuning/solyanka_parted.csv")
train_df = pd.read_csv("drive/MyDrive/ОБЩЕЕ/Отзывус/AI/emotional/data/forfinetuning/best_solyanka.csv")

train_df = pd.concat([train_df] * 3, ignore_index=True)

print(train_df["labels"].value_counts())
train_df

NEGATIVE    4308
POSITIVE    4227
NEUTRAL     4176
Name: labels, dtype: int64


,text,labels
0,Основная проблематика курса включает:\n– Разви...,NEUTRAL
1,Курс «История книги и чтения» знакомит студент...,NEUTRAL
2,Изучение истории немецкой литературы будет про...,NEUTRAL
3,"Данный курс является частью minor \""Франция: и...",NEUTRAL
4,Курс «История французской литературы» построен...,NEUTRAL
...,...,...
12706,"Не надейтесь, что вы такой очень умный и талан...",NEGATIVE
12707,"Итак, жалею, что заканчиваю именно этот вуз. О...",NEGATIVE
12708,Содержание курсов не соответствуют заявленному...,NEGATIVE
12709,Мне так и не удалось поучиться в SkillFactory....,NEGATIVE


In [ ]:
# HMMM
# class_weights = (train_df["labels"].value_counts().sort_index() / max(train_df["labels"].value_counts().sort_index())).values
class_weights = (1 - (train_df["labels"].value_counts().sort_index() - min(train_df["labels"].value_counts().sort_index())) / max(train_df["labels"].value_counts().sort_index())).values
class_weights = ((train_df["labels"].value_counts().sort_index() - min(train_df["labels"].value_counts().sort_index())) / max(train_df["labels"].value_counts().sort_index())).values
class_weights

array([0.03064067, 0.        , 0.01183844])

In [ ]:
class_weights = (1 - (train_df["labels"].value_counts().sort_index() / len(train_df))).values
class_weights

array([0.66108095, 0.67146566, 0.66745339])

In [ ]:
valid_df = pd.read_csv("https://docs.google.com/spreadsheets/d/1kD4JRC_nDcF2ysmXKgYTA6jNpUi3vdbSmJY99lJxZis/export?format=csv", index_col=False)[['text', 'y']].replace({'y' : { -1 : 'NEGATIVE', 0 : 'NEUTRAL', 1 : 'POSITIVE'}})
valid_df.columns = ['text', 'labels']

print(valid_df["labels"].value_counts())
valid_df

NEUTRAL     59
POSITIVE    50
NEGATIVE    34
Name: labels, dtype: int64


,text,labels
0,Электив сам по себе скучноватый.,NEGATIVE
1,"Я не считаю, что полученные знания помогут мне...",NEGATIVE
2,Преподаватель очень тихо ведёт лекцию.,NEUTRAL
3,Преподаватель не пытается как-то ухудшить проц...,NEUTRAL
4,"Ничего не делаем, в конце 2 проекта, можно бра...",NEUTRAL
...,...,...
138,Такая теория никак не поможет ребятам заинтере...,NEGATIVE
139,"Было бы гораздо лучше, будь преподаватель с ка...",NEGATIVE
140,Или это преподавали бы люди с кафедры предприн...,NEUTRAL
141,"Толку учить пользоваться программой, где самый...",NEGATIVE


In [ ]:
from sklearn.model_selection import train_test_split
valid_df_, test_df_ = train_test_split(valid_df, test_size=0.4)
valid_df_ = valid_df_.reset_index(drop=True)
test_df_ = test_df_.reset_index(drop=True)

raw_datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'valid': Dataset.from_pandas(valid_df_),
    'test': Dataset.from_pandas(test_df_)
})

feature_names = ['NEUTRAL', 'POSITIVE', 'NEGATIVE']
raw_datasets['train'] = raw_datasets['train'].cast(Features({'text': Value('string'), 'labels': ClassLabel(num_classes=3, names=feature_names)}))
raw_datasets['valid'] = raw_datasets['valid'].cast(Features({'text': Value('string'), 'labels': ClassLabel(num_classes=3, names=feature_names)}))
raw_datasets['test'] = raw_datasets['test'].cast(Features({'text': Value('string'), 'labels': ClassLabel(num_classes=3, names=feature_names)}))

raw_datasets

Casting the dataset:   0%|          | 0/13 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12711
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 85
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 58
    })
})

In [ ]:
# Проверяем, корретно ли присвоились классы
raw_datasets['train'].features

{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['NEUTRAL', 'POSITIVE', 'NEGATIVE'], id=None)}

### Токенизация собранного датасета

In [ ]:
checkpoint = "blanchefort/rubert-base-cased-sentiment-rurewiews"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding=True, max_length=512)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12711
    })
    valid: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 85
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 58
    })
})

In [ ]:
# Проверяем размер samples
samples = tokenized_datasets['train'][:]
samples = {k: v for k, v in samples.items() if k not in ['idx', 'text']}

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([12711]),
 'input_ids': torch.Size([12711, 512]),
 'token_type_ids': torch.Size([12711, 512]),
 'attention_mask': torch.Size([12711, 512])}

### Настройка параметров и дообучение выбранной модели

Может быть полезно

https://questu.ru/articles/199359/ | https://www.youtube.com/watch?v=HzwM0ClAmOM

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification  # AutoModelForPreTraining ?

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1 = precision_recall_fscore_support(labels, preds, average='macro')[:3]
  return {'precision' : precision, 'recall' : recall, 'F1-score' : f1}

In [ ]:
import torch
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights.astype(np.float32)).to('cuda'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import TrainingArguments

batch_size = 16
epochs_num = 6
checkpoints_path="checkpoints"
# logging_steps = len(raw_datasets['train']) // batch_size

training_args = TrainingArguments(output_dir=checkpoints_path,
                                  overwrite_output_dir=True,
                                  num_train_epochs=epochs_num,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.001,
                                  evaluation_strategy='epoch',
                                  # logging_strategy='epoch',
                                  save_strategy='epoch',
                                  fp16=True
                                  )

trainer = CustomTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

Using cuda_amp half precision backend


In [ ]:
# F1 появится после первой эпохи
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12711
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4770
  Number of trainable parameters = 177855747


Epoch,Training Loss,Validation Loss


### Сохранение модели

In [ ]:
%%capture
!pip install pytest-shutil

In [ ]:
from distutils.dir_util import copy_tree
checkpoint_name = 'checkpoint-3180'
new_model_name = "rubert-base-cased-sentiment-study-feedbacks-solyanka"
save_path = "drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/" + new_model_name

copy_tree(checkpoints_path + '/' + checkpoint_name, save_path)

['drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/config.json',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/pytorch_model.bin',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/tokenizer_config.json',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/special_tokens_map.json',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/vocab.txt',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/tokenizer.json',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/training_args.bin',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/optimizer.pt',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/scheduler.pt',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/scaler.pt',
 'drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/trainer_state.json',
 'drive/MyDrive/rubert-base-cased-sentiment-s

In [ ]:
#trainer.save_model("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/well_its_0.707")

Saving model checkpoint to drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/well_its_0.707
Configuration saved in drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/well_its_0.707/config.json
Model weights saved in drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/well_its_0.707/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/well_its_0.707/tokenizer_config.json
Special tokens file saved in drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/well_its_0.707/special_tokens_map.json


In [ ]:
%%capture
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login() # Сюда надо вставить свой сгенерированный токен [https://huggingface.co/settings/tokens]

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
AutoTokenizer.from_pretrained(save_path).push_to_hub(new_model_name)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
tokenizer config file saved in /tmp/tmptx_7_7jv/tokenizer_config.json
Special tokens file saved in /tmp/tmptx_7_7jv/special_tokens_map.json
Uploading the following files to seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka: tokenizer.json,special_tokens_map.json,tokenizer_config.json,vocab.txt


CommitInfo(commit_url='https://huggingface.co/seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka/commit/67213a434aa849feb5b62fb8c99c1c1f712408af', commit_message='Upload tokenizer', commit_description='', oid='67213a434aa849feb5b62fb8c99c1c1f712408af', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
  # Pytorch
pt_model = AutoModelForSequenceClassification.from_pretrained(save_path)
# pt_model.save_pretrained("drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/finetuned/to_hf/rubert-name-you-picked")
pt_model.push_to_hub(new_model_name)

loading configuration file drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka/config.json
Model config BertConfig {
  "_name_or_path": "drive/MyDrive/rubert-base-cased-sentiment-study-feedbacks-solyanka",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEUTRAL",
    "1": "POSITIVE",
    "2": "NEGATIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEGATIVE": 2,
    "NEUTRAL": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per

KeyboardInterrupt: ignored

In [ ]:
  # TensorFlow
# tf_model = TFDistilBertForSequenceClassification.from_pretrained("path/to/awesome-name-you-picked", from_pt=True)
# tf_model.save_pretrained("path/to/awesome-name-you-picked")

In [ ]:
  # JAX
# flax_model = FlaxDistilBertForSequenceClassification.from_pretrained(
#     "path/to/awesome-name-you-picked", from_pt=True
# )

## Проверка модели

In [ ]:
from transformers import pipeline

In [ ]:
face = pipeline('sentiment-analysis', model=trainer.model.to("cpu"), tokenizer=trainer.tokenizer)

In [ ]:
face = pipeline('sentiment-analysis', model="drive/MyDrive/ОБЩЕЕ/checkpoints_15_12_2022/checkpoint-3975", tokenizer="drive/MyDrive/ОБЩЕЕ/checkpoints_15_12_2022/checkpoint-3975")

In [ ]:
model_name = "seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka" # blanchefort/rubert-base-cased-sentiment-rurewiews
face = pipeline('sentiment-analysis', model=model_name)

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Проверка на дурака

In [ ]:
test = pd.DataFrame({'text': ['Я ненавижу тебя', 'Чай', '...', 'Доброго утра', 'С новым годом', 'Я очень люблю тебя', "здр.", "здр, Тимур. вы можете подойти к директору ИМиКН?"]})
results = face(test['text'].to_list())
print(results)
results = [row['label'] for row in results]
test['pred'] = results
test

[{'label': 'NEGATIVE', 'score': 0.9999551773071289}, {'label': 'POSITIVE', 'score': 0.9982600808143616}, {'label': 'NEUTRAL', 'score': 0.9998340606689453}, {'label': 'NEGATIVE', 'score': 0.7249881625175476}, {'label': 'POSITIVE', 'score': 0.999893069267273}, {'label': 'POSITIVE', 'score': 0.9999675750732422}, {'label': 'NEGATIVE', 'score': 0.9976849555969238}, {'label': 'NEGATIVE', 'score': 0.9999479055404663}]


,text,pred
0,Я ненавижу тебя,NEGATIVE
1,Чай,POSITIVE
2,...,NEUTRAL
3,Доброго утра,NEGATIVE
4,С новым годом,POSITIVE
5,Я очень люблю тебя,POSITIVE
6,здр.,NEGATIVE
7,"здр, Тимур. вы можете подойти к директору ИМиКН?",NEGATIVE


In [ ]:
# LABEL ТУТ, чтобы долго в голове не держать

labels = { 'NEGATIVE' : 'NGTV', 'NEUTRAL' : 'NEUT', 'POSITIVE' : 'PSTV'}
# labels = { 'LABEL_2' : 'NGTV', 'LABEL_0' : 'NEUT', 'LABEL_1' : 'PSTV'}

In [ ]:
test_df = pd.read_csv("https://docs.google.com/spreadsheets/d/1kD4JRC_nDcF2ysmXKgYTA6jNpUi3vdbSmJY99lJxZis/export?format=csv")[['text', 'y', 'y_alter']].replace({'y' : { -1 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}}).replace({'y_alter' : { -1 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}})
test_df.columns = ['text', 'true', 'possible']

In [ ]:
%%time
test_df['pred'] = [row['label'] for row in face(test_df['text'].to_list())]
test_df = test_df.replace({'pred' : labels})
test_df

CPU times: user 16.5 s, sys: 32.3 ms, total: 16.5 s
Wall time: 16.9 s


,text,true,possible,pred
0,Электив сам по себе скучноватый.,NGTV,NEUT,NEUT
1,"Я не считаю, что полученные знания помогут мне...",NGTV,NaN,NEUT
2,Преподаватель очень тихо ведёт лекцию.,NEUT,NaN,NGTV
3,Преподаватель не пытается как-то ухудшить проц...,NEUT,NaN,NGTV
4,"Ничего не делаем, в конце 2 проекта, можно бра...",NEUT,PSTV,NGTV
...,...,...,...,...
138,Такая теория никак не поможет ребятам заинтере...,NGTV,NaN,NGTV
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV,NEUT,PSTV
140,Или это преподавали бы люди с кафедры предприн...,NEUT,NaN,NGTV
141,"Толку учить пользоваться программой, где самый...",NGTV,NaN,NGTV


### Допоміжна функція

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as alotmetrics

def custom_f1(y_pred, y_true, y_possible=None, mode='fact', average='macro'):
  y_min = []
  y_max = []
  for i in range(len(y_pred)):
    if (not(pd.isnull(y_possible[i]))):
      if (y_pred[i] == y_true[i]):
        y_min.append(y_possible[i])
        y_max.append(y_true[i])
      else:
        y_min.append(y_true[i])
        y_max.append(y_possible[i])
    else:
      y_max.append(y_true[i])
      y_min.append(y_true[i])

  if (mode == 'fact'):
    return alotmetrics(y_true, y_pred, average=average)[:3]
  elif (mode == 'min'):
    return alotmetrics(y_min, y_pred, average=average)[:3]
  elif (mode == 'max'):
    return alotmetrics(y_max, y_pred, average=average)[:3]
  elif (mode == 'to table'):
    # fact min max
    return ("{}\t{}\t{}".format(alotmetrics(y_true, y_pred, average=average)[2], alotmetrics(y_min, y_pred, average=average)[2], alotmetrics(y_max, y_pred, average=average)[2]))
  else:
    print("Допустимые режимы: min/fact/max")
    return

### Момент истины

In [ ]:
sum(test_df['true'] == test_df['pred']) / len(test_df['true'])

0.7272727272727273

In [ ]:
# print(checkpoint)
print("Precision\tRecall\tF1-score")
print("Min: {}".format(custom_f1(test_df['pred'], test_df['true'], test_df['possible'], 'min')))
print("Fact: {}".format(custom_f1(test_df['pred'], test_df['true'], test_df['possible'], 'fact')))
print("Max: {}".format(custom_f1(test_df['pred'], test_df['true'], test_df['possible'], 'max')))

Precision	Recall	F1-score
Min: (0.6, 0.6342251950947603, 0.6040747057041731)
Fact: (0.7164021164021164, 0.7313060817547358, 0.7158325194108187)
Max: (0.7841269841269841, 0.7931916790999982, 0.7845454545454545)


In [ ]:
print(checkpoint)
print(custom_f1(test_df['pred'], test_df['true'], test_df['possible'], 'to table'))

blanchefort/rubert-base-cased-sentiment-rurewiews
0.7072756518161074	0.604040404040404	0.7842005489064313
